# Validation of Gridpoint and Spectral LAM templates: data encoding

The objective is here to encode and decode the same data using a modified version of ECCODES and WGRIB2.

## environment

based on Eccodes 2.6.0 (branch feature/MeteoFrance_lamOMM) and a modified version of wgrib2 2.0.5

In [1]:
export TESTHOME=/home/yann.genin/lamOMM
export WGRIB2_PREFIX=$TESTHOME/wgrib2/grib2/wgrib2/
export GRIB_API_CMAKE_PREFIX=$TESTHOME/eccodes_build
export GRIB_API_PREFIX=$GRIB_API_CMAKE_PREFIX
export GRIB_API_SOURCE_PREFIX=$TESTHOME/eccodes-2.6.0-Source

In [2]:
export WGRIB2=$WGRIB2_PREFIX/wgrib2
export GRIB_DUMP=$GRIB_API_PREFIX/bin/grib_dump
export GRIB_DEFINITION_PATH=$GRIB_API_SOURCE_PREFIX/definitions
export GRIB_SAMPLES_PATH=$GRIB_API_SOURCE_PREFIX/samples
export LD_LIBRARY_PATH=$GRIB_API_PREFIX/lib

export ECCODES_DEBUG=0

## Grid point data

compilation of program using Eccodes

In [3]:
cat lam_gp.c

#include <stdlib.h>
#include <math.h>
#include <stdio.h>
#include <string.h>

#include "grib_api.h"

/*
 * Test encoding of grid-point LAM fields
 * Test FA conversion to grib_api
 * v1.0 philippe.marguinaud 02/2016
 * v1.01 yann.genin 11/2018
 */



int main (int argc, char * argv[])
{
  grib_handle * h;
  size_t len;
  const char * grids[] = {"lambert_lam", "mercator_lam", "polar_stereographic_lam"};
  int igrid;
  const char * f_text = argv[1];
  double * values = NULL;
  FILE * fp;
  size_t size, nlon, nlat;
  int i;


  fp = fopen (f_text, "r");
  fscanf (fp, "%ld %ld\n", &nlon, &nlat);
  len = nlon * nlat;
  values = (double *)malloc (sizeof (double) * len);
  for (i = 0; i < len; i++)
    fscanf (fp, "%lf\n", &values[i]);
  fclose (fp);



  for (igrid = 0; igrid < 3; igrid++)
    {
       GRIB_CHECK (((h = grib_handle_new_from_samples (NULL, "reduced_gg_ml_grib2")) == NULL), 0);
       len = strlen ("lambert_lam");
       GRIB_CHECK (grib_set_string (h, "gridType", "lambert_lam

In [4]:
cc -g -o lam_gp.x lam_gp.c -I$GRIB_API_PREFIX/include -L$GRIB_API_PREFIX/lib -leccodes -Wl,-rpath,$GRIB_API_PREFIX/lib

### Grid point data: metadata encoding using Eccodes
Wgrib2 encoding of metadata was not implemented.


In [5]:
./lam_gp.x lam_gp.data

### Grid point data: Comparison of section 3 between Eccodes and Wgrib2 for grid point templates
#### Lambert

In [6]:
$GRIB_DUMP -O lam_gp_lambert_lam.grib > lam_gp_lambert_lam.grib.grib_api.txt 2>/dev/null
cat lam_gp_lambert_lam.grib.grib_api.txt |awk '/SECTION_3/{f=1}/SECTION_4/{f=0}f'

======================   SECTION_3 ( length=97, padding=0 )    ======================
1-4       section3Length = 97
5         numberOfSection = 3
6         sourceOfGridDefinition = 0 [Specified in Code table 3.1 (grib2/tables/4/3.0.table) ]
7-10      numberOfDataPoints = 4096
11        numberOfOctectsForNumberOfPoints = 0
12        interpretationOfNumberOfPoints = 0 [There is no appended list (grib2/tables/4/3.11.table) ]
13-14     gridDefinitionTemplateNumber = 33 [Unknown code table entry (grib2/tables/4/3.1.table) ]
15        shapeOfTheEarth = 6 [Earth assumed spherical with radius of 6,371,229.0 m (grib2/tables/4/3.2.table) ]
16        scaleFactorOfRadiusOfSphericalEarth = MISSING
17-20     scaledValueOfRadiusOfSphericalEarth = MISSING
21        scaleFactorOfEarthMajorAxis = MISSING
22-25     scaledValueOfEarthMajorAxis = MISSING
26        scaleFactorOfEarthMinorAxis = MISSING
27-30     scaledValueOfEarthMinorAxis = MISSING
31-34     Nx = 64
35-38     Ny = 64
39-42     latitudeOfFi

In [7]:
$WGRIB2 -grid lam_gp_lambert_lam.grib

1:0:grid_template=33:
	Lambert Conformal with LAM extension (U=[53,53],C=[8,8]): (64 x 64) input (null) output WE:SN res -1
	Lat1 66.768495 Lon1 25.158021 LoV 26.640000
	LatD 67.360000 Latin1 67.360000 Latin2 67.360000
	LatSP 0.000000 LonSP 0.000000
	North Pole (64 x 64) Dx 2500.000000 m Dy 2500.000000 m mode -1
	stagger 15 offset(even_x:dx/2 odd_x:dx/2 y:dy/2) storage trim


#### Mercator

In [8]:
$GRIB_DUMP -O lam_gp_mercator_lam.grib > lam_gp_mercator_lam.grib.grib_api.txt 2>/dev/null
cat lam_gp_mercator_lam.grib.grib_api.txt |awk '/SECTION_3/{f=1}/SECTION_4/{f=0}f'

======================   SECTION_3 ( length=88, padding=0 )    ======================
1-4       section3Length = 88
5         numberOfSection = 3
6         sourceOfGridDefinition = 0 [Specified in Code table 3.1 (grib2/tables/4/3.0.table) ]
7-10      numberOfDataPoints = 4096
11        numberOfOctectsForNumberOfPoints = 0
12        interpretationOfNumberOfPoints = 0 [There is no appended list (grib2/tables/4/3.11.table) ]
13-14     gridDefinitionTemplateNumber = 13 [Unknown code table entry (grib2/tables/4/3.1.table) ]
15        shapeOfTheEarth = 6 [Earth assumed spherical with radius of 6,371,229.0 m (grib2/tables/4/3.2.table) ]
16        scaleFactorOfRadiusOfSphericalEarth = MISSING
17-20     scaledValueOfRadiusOfSphericalEarth = MISSING
21        scaleFactorOfEarthMajorAxis = MISSING
22-25     scaledValueOfEarthMajorAxis = MISSING
26        scaleFactorOfEarthMinorAxis = MISSING
27-30     scaledValueOfEarthMinorAxis = MISSING
31-34     Ni = 64
35-38     Nj = 64
39-42     latitudeOfFi

In [9]:
$WGRIB2 -grid lam_gp_mercator_lam.grib

1:0:grid_template=13:
	Mercator grid with LAM extension (U=[53,53],C=[8,8]): (64 x 64) LatD 0.000000 input (null) output WE:SN res -1
	lat 67.937201 to 72.937201 by 0.000000 m
	lon 25.158021 to 29.158021 by 0.000000 m
	orientation 0.000000
	stagger 15 offset(even_x:dx/2 odd_x:dx/2 y:dy/2) storage trim


#### Polar stereographic

In [10]:
$GRIB_DUMP -O lam_gp_polar_stereographic_lam.grib > lam_gp_polar_stereographic_lam.grib.grib_api.txt 2>/dev/null
cat lam_gp_polar_stereographic_lam.grib.grib_api.txt |awk '/SECTION_3/{f=1}/SECTION_4/{f=0}f'

======================   SECTION_3 ( length=81, padding=0 )    ======================
1-4       section3Length = 81
5         numberOfSection = 3
6         sourceOfGridDefinition = 0 [Specified in Code table 3.1 (grib2/tables/4/3.0.table) ]
7-10      numberOfDataPoints = 4096
11        numberOfOctectsForNumberOfPoints = 0
12        interpretationOfNumberOfPoints = 0 [There is no appended list (grib2/tables/4/3.11.table) ]
13-14     gridDefinitionTemplateNumber = 23 [Unknown code table entry (grib2/tables/4/3.1.table) ]
15        shapeOfTheEarth = 6 [Earth assumed spherical with radius of 6,371,229.0 m (grib2/tables/4/3.2.table) ]
16        scaleFactorOfRadiusOfSphericalEarth = MISSING
17-20     scaledValueOfRadiusOfSphericalEarth = MISSING
21        scaleFactorOfEarthMajorAxis = MISSING
22-25     scaledValueOfEarthMajorAxis = MISSING
26        scaleFactorOfEarthMinorAxis = MISSING
27-30     scaledValueOfEarthMinorAxis = MISSING
31-34     Nx = 64
35-38     Ny = 64
39-42     latitudeOfFi

In [11]:
$WGRIB2 -grid lam_gp_polar_stereographic_lam.grib

1:0:grid_template=23:
	polar stereographic grid with LAM extension (U=[53,53],C=[8,8]): (64 x 64) input (null) output WE:SN res -1
	South pole lat1 67.937201 lon1 25.158021 latD 0.000000 lonV 0.000000 dx 2500.000000 m dy 2500.000000 m


## Spectral data

compilation of program using Eccodes

In [12]:
cat lam_bf.c

#include <stdlib.h>
#include <math.h>
#include <stdio.h>
#include <string.h>

#include "grib_api.h"

/*
 * Test encoding of LAM spectral fields
 * v1.0 philippe.marguinaud 02/2016
 * v1.01 yann.genin 11/2018
 */


#define ILCHAM 3128 
#define NSTRON   10
#define NSMAX    31
#define NMSMAX   31

int main (int argc, char * argv[])
{
  grib_handle * h;
  size_t len;
  const char * grids[] = {"lambert_bf", "mercator_bf", "polar_stereographic_bf"};
  int igrid;
  const char * f_text = argv[1];
  double * values = NULL;
  FILE * fp;
  size_t dum;
  int i;

  
  fp = fopen (f_text, "r");
  fscanf (fp, "%d %d\n", &len, &dum);
  values = (double *)malloc (sizeof (double) * len);
  for (i = 0; i < len; i++)
    fscanf (fp, "%lf\n", &values[i]);
  fclose (fp);

  for (igrid = 0; igrid < 3; igrid++)
    {
      GRIB_CHECK (((h = grib_handle_new_from_samples (NULL, "lambert_bf_grib2")) == NULL), 0);
      GRIB_CHECK (grib_set_long (h, "centre", 85), 0);
      len = strlen (grids[igrid]);
      GRIB

In [13]:
cc -g -o lam_bf.x lam_bf.c -I$GRIB_API_PREFIX/include -L$GRIB_API_PREFIX/lib -leccodes -Wl,-rpath,$GRIB_API_PREFIX/lib


### Spectral data: metadata encoding using Eccodes
Wgrib2 encoding of metadata was not implemented.


In [14]:
cp lam_bf.data.gz lam_bf.txt.gz
gunzip lam_bf.txt.gz
./lam_bf.x lam_bf.txt

### Spectral data: metadata decoding
#### lambert 

In [15]:
$GRIB_DUMP -O lam_bf_lambert_bf.grib > lam_bf_lambert_bf.grib.grib_api.txt 2>/dev/null
cat lam_bf_lambert_bf.grib.grib_api.txt  |awk '/SECTION_3/{f=1}/SECTION_4/{f=0}f'

======================   SECTION_3 ( length=121, padding=0 )   ======================
1-4       section3Length = 121
5         numberOfSection = 3
6         sourceOfGridDefinition = 0 [Specified in Code table 3.1 (grib2/tables/4/3.0.table) ]
7-10      numberOfDataPoints = 3128
11        numberOfOctectsForNumberOfPoints = 0
12        interpretationOfNumberOfPoints = 0 [There is no appended list (grib2/tables/4/3.11.table) ]
13-14     gridDefinitionTemplateNumber = 63 [Unknown code table entry (grib2/tables/4/3.1.table) ]
15        spectralType = 99 [Unknown code table entry (grib2/tables/4/3.6.table) ]
16-19     biFourierResolutionParameterN = 31
20-23     biFourierResolutionParameterM = 31
24        biFourierTruncationType = 99 [Unknown code table entry () ]
25-32     Lx = 2000000
33-40     Lux = 1800000
41-48     Lcx = 100000
49-56     Ly = 2000000
57-64     Luy = 1800000
65-72     Lcy = 100000
73        shapeOfTheEarth = 6 [Earth assumed spherical with radius of 6,371,229.0 m (grib2/

In [16]:
$WGRIB2 -grid lam_bf_lambert_bf.grib

1:0:grid_template=63:Spectral Lambert 0 N=31 M=31, code_table_3.6=99 code_table_3.25=99
	Subdomains definition lx=2000000 lux=1800000 lcx=100000 ly=2000000 luy=1800000 lcy=100000
	South Pole latin1 67360000 latin2 67360000 latSP 0 lonSP 0


#### Mercator

In [17]:
$GRIB_DUMP -O lam_bf_mercator_bf.grib > lam_bf_mercator_bf.grib.grib_api.txt 2>/dev/null
cat lam_bf_mercator_bf.grib.grib_api.txt |awk '/SECTION_3/{f=1}/SECTION_4/{f=0}f'

======================   SECTION_3 ( length=112, padding=0 )   ======================
1-4       section3Length = 112
5         numberOfSection = 3
6         sourceOfGridDefinition = 0 [Specified in Code table 3.1 (grib2/tables/4/3.0.table) ]
7-10      numberOfDataPoints = 3128
11        numberOfOctectsForNumberOfPoints = 0
12        interpretationOfNumberOfPoints = 0 [There is no appended list (grib2/tables/4/3.11.table) ]
13-14     gridDefinitionTemplateNumber = 61 [Unknown code table entry (grib2/tables/4/3.1.table) ]
15        spectralType = 99 [Unknown code table entry (grib2/tables/4/3.6.table) ]
16-19     biFourierResolutionParameterN = 31
20-23     biFourierResolutionParameterM = 31
24        biFourierTruncationType = 99 [Unknown code table entry () ]
25-32     Lx = 2000000
33-40     Lux = 1800000
41-48     Lcx = 100000
49-56     Ly = 2000000
57-64     Luy = 1800000
65-72     Lcy = 100000
73        shapeOfTheEarth = 6 [Earth assumed spherical with radius of 6,371,229.0 m (grib2/

In [18]:
$WGRIB2 -grid lam_bf_mercator_bf.grib

1:0:grid_template=61:Spectral Mercator LaD 0 N=31 M=31, code_table_3.6=99 code_table_3.25=99
	Subdomains definition lx=2000000 lux=1800000 lcx=100000 ly=2000000 luy=1800000 lcy=100000
	lat 67937201 to 72937201 
	lon 25158021 to 29158021 
	orientation 0.000000


#### Polar stereographic

In [19]:
$GRIB_DUMP -O lam_bf_polar_stereographic_bf.grib > lam_bf_polar_stereographic_bf.grib.grib_api.txt 2>/dev/null
cat lam_bf_polar_stereographic_bf.grib.grib_api.txt |awk '/SECTION_3/{f=1}/SECTION_4/{f=0}f'

======================   SECTION_3 ( length=106, padding=0 )   ======================
1-4       section3Length = 106
5         numberOfSection = 3
6         sourceOfGridDefinition = 0 [Specified in Code table 3.1 (grib2/tables/4/3.0.table) ]
7-10      numberOfDataPoints = 3128
11        numberOfOctectsForNumberOfPoints = 0
12        interpretationOfNumberOfPoints = 0 [There is no appended list (grib2/tables/4/3.11.table) ]
13-14     gridDefinitionTemplateNumber = 62 [Unknown code table entry (grib2/tables/4/3.1.table) ]
15        spectralType = 99 [Unknown code table entry (grib2/tables/4/3.6.table) ]
16-19     biFourierResolutionParameterN = 31
20-23     biFourierResolutionParameterM = 31
24        biFourierTruncationType = 99 [Unknown code table entry () ]
25-32     Lx = 2000000
33-40     Lux = 1800000
41-48     Lcx = 100000
49-56     Ly = 2000000
57-64     Luy = 1800000
65-72     Lcy = 100000
73        shapeOfTheEarth = 6 [Earth assumed spherical with radius of 6,371,229.0 m (grib2/

In [20]:
$WGRIB2 -grid lam_bf_polar_stereographic_bf.grib

1:0:grid_template=62:Spectral Polar Stereographic 0 N=31 M=31, code_table_3.6=99 code_table_3.25=99
	Subdomains definition lx=2000000 lux=1800000 lcx=100000 ly=2000000 luy=1800000 lcy=100000
	pole 0 lat1 67937201 lon1 25158021 latD 0 lonV 0


## cleanup

In [21]:
rm -f *.grib *.txt lam_*.x